In [ ]:
!pip install numpy matplotlib torch diffusers transformers
!pip install opensmile  # For audio feature extraction
!pip install sounddevice
!pip install pydub
!apt-get install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (319 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 124947 files and directories currently installed.)
Pre

In [ ]:
!pip install pyaudio  # For audio recording

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp311-cp311-linux_x86_64.whl size=67392 sha256=01ff79ed33289cf0ecd1f06c080eb6d5d2aaa01ba0db53b36487a6bf2627e4b9
  Stored in directory: /root/.cache/pip/wheels/80/b1/c1/67e4ef443de2665d86031d4760508094eab5de37d5d64d9c27
Successfully built pyaudio


In [ ]:
import sounddevice as sd
import numpy as np
import wave

def record_audio(filename, duration=5, samplerate=44100):
    print("Recording...")
    audio_data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
    sd.wait()  # Wait until recording is finished
    print("Recording finished.")

    # Save the audio data to a WAV file
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(2)
        wf.setsampwidth(2)
        wf.setframerate(samplerate)
        wf.writeframes(audio_data.tobytes())

In [ ]:
from scipy.io import wavfile

def read_audio_file(filename):
    sample_rate, audio_data = wavfile.read(filename)
    print(f"Loaded audio: {audio_data.shape} shape, {sample_rate} Hz")
    return audio_data, sample_rate


In [ ]:
import opensmile

def extract_audio_features(audio_file):
    # Initialize openSMILE with the ComParE_2016 feature set
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.Functionals
    )

    # Process the audio file and extract features
    features = smile.process_file(audio_file)
    return features

In [ ]:
def map_features_to_visual_params(features):
    """
    Maps extracted audio features to visual parameters for art generation.

    Returns:
        dict: A dictionary of visual parameters
    """
    # This is a simplified mapping

    # Extract some key features
    energy = features.get('pcm_RMSenergy_sma_max', 0.5)  # Energy/volume
    pitch = features.get('F0_sma_mean', 0.5)  # Pitch
    tempo = features.get('pcm_zcr_sma_mean', 0.5)  # Zero crossing rate (related to tempo)

    # Map to visual parameters
    visual_params = {
        'color_intensity': min(1.0, energy * 2),  # Higher energy = more vibrant colors
        'stroke_width': 1 + (energy * 5),  # Higher energy = thicker strokes
        'complexity': 0.3 + (pitch * 0.7),  # Higher pitch = more complex patterns
        'motion_blur': tempo * 3,  # Higher tempo = more motion blur
    }

    return visual_params


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

def generate_art(visual_params, output_path="generated_art.png"):
    """
    Generate abstract art based on the provided visual parameters.
    """
    # Initialize the Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4",
        torch_dtype=torch.float16
    )
    pipe = pipe.to("cuda")

    # Construct a prompt based on visual parameters
    color_intensity = "vibrant" if visual_params['color_intensity'] > 0.6 else "muted"
    complexity = "complex" if visual_params['complexity'] > 0.6 else "simple"
    motion = "dynamic" if visual_params['motion_blur'] > 1.5 else "static"

    prompt = f"Abstract art, {color_intensity} colors, {complexity} patterns, {motion} brushstrokes"

    # Generate the image
    image = pipe(prompt).images[0]

    # Save the image
    image.save(output_path)

    return image


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
def generate_art_with_lora(visual_params, output_path="generated_art.png"):
    """
    Generate abstract art based on the provided visual parameters using LoRA fine-tuned model.
    """
    # Initialize the Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16
    )
    pipe = pipe.to("cuda")

    # Load the LoRA weights
    lora_weights_path = "/content/drive/MyDrive/lora_weights.pt"  # Path to your saved LoRA weights
    lora_state_dict = torch.load(lora_weights_path)

    # Apply the LoRA weights to the UNet
    for name, param in pipe.unet.named_parameters():
        if name in lora_state_dict:
            param.data = lora_state_dict[name].to(param.device)

    # Construct a prompt based on visual parameters
    color_intensity = "vibrant" if visual_params['color_intensity'] > 0.6 else "muted"
    complexity = "complex" if visual_params['complexity'] > 0.6 else "simple"
    motion = "dynamic" if visual_params['motion_blur'] > 1.5 else "static"

    prompt = f"Abstract art, {color_intensity} colors, {complexity} patterns, {motion} brushstrokes"

    # Generate the image
    image = pipe(prompt, num_inference_steps=50).images[0]

    image.save(output_path)

    return image


In [ ]:
from pydub import AudioSegment

file_name = "uptown.mp3"
audio = AudioSegment.from_mp3(file_name)
wav_file_name = file_name.replace('.mp3', '.wav')
audio.export(wav_file_name, format='wav')
print(f'Converted {file_name} to {wav_file_name}')

Converted uptown.mp3 to uptown.wav


In [ ]:
def main():
    # 1. Use a pre-recorded WAV file
    audio_file = wav_file_name  # Replace with your file path

    # 2. Read the audio file
    audio_data, sample_rate = read_audio_file(audio_file)

    # 3. Extract features using openSMILE
    try:
        features = extract_audio_features(audio_file)

        # 4. Map features to visual parameters
        visual_params = map_features_to_visual_params(features)

        # 5. Generate art
        generated_image = generate_art(visual_params)

        print("Art generation complete!")

    except Exception as e:
        print(f"Error during processing: {e}")

if __name__ == "__main__":
    main()


Loaded audio: (6529536, 2) shape, 48000 Hz


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Art generation complete!


In [ ]:
def main():
    # 1. Use a pre-recorded WAV file
    audio_file = wav_file_name  # Replace with your file path

    # 2. Extract features using openSMILE
    try:
        features = extract_audio_features(audio_file)

        # 3. Map features to visual parameters
        visual_params = map_features_to_visual_params(features)

        # 4. Generate art using LoRA fine-tuned model
        generated_image = generate_art_with_lora(visual_params, "lora_generated_art.png")

        print("Art generation with LoRA complete!")

    except Exception as e:
        print(f"Error during processing: {e}")


In [ ]:
if __name__ == "__main__":
    main()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

<ipython-input-16-00bda1e025df>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_state_dict = torch.load(lora_weights_path)


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/diffusers/image_processor.py:147: RuntimeWarning: invalid value encountered in cast
  images = (images * 255).round().astype("uint8")
Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


Art generation with LoRA complete!
